In [1]:
import pandas as pd
import numpy as np
import torch 
import torch.nn.functional as F
from transformers import XLNetTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

In [2]:
column = ['type','title','text']
train = pd.read_csv('./data_after_sep/train.tsv',sep='\t',names=column)
test =pd.read_csv('./data_after_sep/dev.tsv',sep='\t',names=column)

In [3]:
train_x = train['text']
train_y = train['type']
train_x = np.array(train_x.tolist())
train_y = np.array(train_y)
print(train_x.shape)
print(train_y.shape)

(25546,)
(25546,)


In [4]:
test_x = test['text']
test_y = test['type']
test_x = np.array(test_x.tolist())
test_y = np.array(test_y)
print(test_x.shape)
print(test_y.shape)

(5000,)
(5000,)


In [5]:
MAX_LENGTH = 512
NUM_LABELS = 7
tokenizer = XLNetTokenizer.from_pretrained('./chinese_xlnet_mid_pytorch/')
input_dict = tokenizer.batch_encode_plus(train_x,
                                         add_special_tokens = True,
                                         max_length = MAX_LENGTH,
                                         return_special_tokens_mask = True,
                                         pad_to_max_length = True,
                                         return_tensors = 'pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [6]:
class TrainDataset(Dataset):
    def __init__(self,input_dict,y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return inputid , tokentype , attentionmask , y
    
    def __len__(self):
        return len(self.input_ids)
    
class TestDataset(Dataset):
    def __init__(self, input_dict):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
       
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        return inputid , tokentype , attentionmask 
    
    def __len__(self):
        return len(self.input_ids)
    

In [7]:
BATCH_SIZE = 2
trainset = TrainDataset(input_dict,train_y)
trainloader = DataLoader(trainset,batch_size = BATCH_SIZE)

In [8]:
test_dict = tokenizer.batch_encode_plus(test_x,
                                         add_special_tokens = True,
                                         max_length = MAX_LENGTH,
                                         return_special_tokens_mask = True,
                                         pad_to_max_length = True,
                                         return_tensors = 'pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [9]:
BATCH_SIZE = 64
testset = TrainDataset(test_dict,test_y)
testloader = DataLoader(testset,batch_size = BATCH_SIZE)

In [10]:
from transformers import XLNetForSequenceClassification

model = XLNetForSequenceClassification.from_pretrained('./chinese_xlnet_mid_pytorch/',num_labels = NUM_LABELS)

Some weights of the model checkpoint at ./chinese_xlnet_mid_pytorch/ were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at ./chinese_xlnet_mid_pytorch/ and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to 

In [11]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    predictions_withoutmax = None
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            tokens, segments, masks = data[:3]
            outputs = model(input_ids=tokens, 
                            token_type_ids=segments, 
                            attention_mask=masks)
            
            logits = outputs[0]
            after_softmax = F.softmax(logits.data, dim=1)
            _, pred = torch.max(after_softmax, 1)
            
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc

In [12]:
from torch.autograd import Variable

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)

EPOCHS = 5
for epoch in range(EPOCHS):
    step = 0
    running_loss = 0
    total = 0
    correct = 0
    for data in trainloader:
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
        optimizer.zero_grad()
        outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors, 
                            labels=labels)
        loss = outputs[0]
        pred = outputs[1]
        total += pred.size()[0]
        pred = torch.argmax(pred,dim=-1)
        correct += (pred==labels).sum().item()

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    torch.save(model.state_dict(),'./xlnet_state_dict'+str(epoch)+'.pkl')
    prediction, acc = get_predictions(model, trainloader, compute_acc=True)
    test_pred,test_acc = get_predictions(model, testloader, compute_acc=True)
    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))
    print(test_acc)

cuda:0


/usr/local/lib/python3.6/dist-packages/transformers/modeling_xlnet.py:283: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorIterator.cpp:918.)
  attn_score = (ac + bd + ef) * self.scale


RuntimeError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 10.76 GiB total capacity; 7.98 GiB already allocated; 112.81 MiB free; 9.42 GiB reserved in total by PyTorch)

In [12]:
BATCH_SIZE = 64
testset = TrainDataset(test_dict,test_y)
testloader = DataLoader(testset,batch_size = BATCH_SIZE)
NUM_LABELS = 7
from transformers import XLNetForSequenceClassification
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = XLNetForSequenceClassification.from_pretrained('./chinese_xlnet_mid_pytorch/',num_labels = NUM_LABELS)
model = model.to(device)
model = torch.load('XLNet_selftraining_4_0.pkl')
model.eval()
test_pred,test_acc = get_predictions(model, testloader, compute_acc=True)
print(test_acc)

Some weights of the model checkpoint at ./chinese_xlnet_mid_pytorch/ were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at ./chinese_xlnet_mid_pytorch/ and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to 

0.8328
